<a href="https://colab.research.google.com/github/tankien76/CS114.K21/blob/master/Captone_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Danh sách nhóm
- Đoàn Xuân Minh - MSSV: 18521093
- Lê Trần Phúc Nguyên - MSSV: 18521157
- Phan Quang Tấn - MSSV: 18521377


# Mô tả bài toán : Dự đoán chủ đề của một bài báo tiếng Anh
- Input: Một đoạn văn bản của một bài báo (article)
- Output: Chủ đề của đoạn văn bản 

Code tham khảo: https://towardsdatascience.com/sarcasm-detection-step-towards-sentiment-analysis-84cb013bb6db

Code tham khảo: https://pythonspot.com/nltk-stop-words/

# Prepare dataset

Thực hiện cào dữ liệu (crawl data) từ các trang báo tiếng Anh nổi tiếng như : Thesun https://www.thesun.co.uk/ , Dailymail https://www.dailymail.co.uk/home/index.html, Telepraph http://telegraph.co.uk/

Code Crawl : https://github.com/tankien76/CS114.K21/blob/master/CrawlerTest.ipynb

Tập dataset sau khi crawl gồm có hơn 2600 bài báo với 8 chủ đề khác nhau như : sport, tvshowbiz, femail,...

In [1]:
!pip install PyDrive

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
downloaded = drive.CreateFile({'id':"1o2zIVZBmqSJa8huDq_Nn8waeTksbH4KI"})
downloaded.GetContentFile('text_data.csv')

Import các thư viện:
- pandas : đọc dữ liệu file json
- numpy : xử lý dữ liệu text

In [5]:
import pandas as pd, numpy as np, re

Đọc dữ liệu bằng thư viện pandas từ file text_data.csv

In [6]:
dataset = pd.read_csv("text_data.csv", encoding='ISO 8859-1')
dataset.head()

,article,category
0,PHOTOS of McDonalds restaurants and merchandis...,lifestyle
1,"ARIESMarch 21 to April 20SATURN, the most powe...",lifestyle
2,BRITS are being tempted to fly abroad once loc...,lifestyle
3,DONNA Mitchell tried every fad diet in the boo...,lifestyle
4,FASHION fans can buy jeans and a sleeveless to...,lifestyle


Đưa dữ liệu article và category vào list tương ứng

In [7]:
features = dataset['article']
labels = dataset['category']

Thực hiện chuyển đổi các nhãn dán thành số để dễ dàng thực hiện trong khi training và test model

In [8]:
from sklearn import preprocessing
number = preprocessing.LabelEncoder()
labels= number.fit_transform(labels)

# Feature Engineering
Tiến hành xử lý dữ liệu văn bản (article) :

- Loại bỏ các ký tự đặc biệt(dấu câu, số)
- Loại bỏ các stopword trong tiếng anh
- Vector hóa dữ liệu

download punkt trong thư viện nltk để tách từ khi xử lý dữ liệu

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

download stopwords trong thư viện nltk để xử lý các từ không cần thiết trong tiếng anh

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Import thư viện nltk để sử lý ngôn ngữ tự nhiên

In [11]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

Loại bỏ các ký tự đặc biệt như: dấu câu, số,...

In [12]:
features = features.apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

Loại bỏ các stopword (các từ không ảnh hưởng đến nội dung của văn bản) trong tiếng Anh

In [13]:
stop_words = set(stopwords.words('english'))
for i in range (len(features)):
  word_tokens = word_tokenize(features[i])

  filtered_sentence = [w for w in word_tokens if not w in stop_words]

  filtered_sentence = []

  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence.append(w)

Vector hóa dữ liệu văn bản bằng TF-IDF với max_feature = 5000 (dùng để tranh ngốn ram của máy):
- TF: tần suất xuất hiện của 1 từ trong 1 văn bản
- IDF: tần suất nghịch của 1 từ trong 1 tập văn bản
- .fit: Máy học từ vựng và tính toán idf( tần suất nghịch của 1 từ trong tập các văn bản) của train set.

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfIdfVec = TfidfVectorizer()
tfIdfVec.fit(features)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

Đưa dữ liệu văn bản về matrix để training model

In [15]:
tf_idf_x = tfIdfVec.transform(features)

Tiến hành chia dữ liệu train và test: theo tỉ lệ 90% training và 10% testing vì khi thực hiện theo tỉ lệ 80/20 hoặc 70/30 thì độ chính xác của các thuật toán giảm

In [16]:
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(tf_idf_x.toarray(), labels, test_size = .1, random_state = 0)

# Chosing Model

Thêm thư viện sklearn gồm :
- Các model sử dụng : LinearSVC, Multinomial Naive Bayes, Logistic Regresstion, Random Forest

In [17]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [18]:
lsvc = LinearSVC()
lsvc.fit(features_train, labels_train)
print("Train accuracy: ",lsvc.score(features_train, labels_train))
print("Test accuracy: ",lsvc.score(features_test, labels_test))

Train accuracy:  0.9970501474926253
Test accuracy:  0.9204545454545454


In [19]:
mnb = MultinomialNB()
mnb.fit(features_train, labels_train)
print("train accuracy: ",mnb.score(features_train, labels_train))
print("Test accuracy: ",mnb.score(features_test, labels_test))

train accuracy:  0.9157184997892962
Test accuracy:  0.8636363636363636


In [20]:
lr = LogisticRegression()
lr.fit(features_train, labels_train)
print("train accuracy: ",lr.score(features_train, labels_train))
print("Test accuracy: ",lr.score(features_test, labels_test))

train accuracy:  0.9637589549093973
Test accuracy:  0.8901515151515151


In [21]:
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print("train accuracy: ",rfc.score(features_train, labels_train))
print("Test accuracy: ",rfc.score(features_test, labels_test))

train accuracy:  0.9953645174884113
Test accuracy:  0.7234848484848485


In [22]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [23]:
y_pred=lsvc.predict(features_test)
f1 = f1_score(labels_test, y_pred, average='macro')
print("F1 score: ",f1)

F1 score:  0.913350084588004


In [24]:
y_pred=mnb.predict(features_test)
f1 = f1_score(labels_test, y_pred, average='macro')
print("F1 score: ",f1)

F1 score:  0.7649294130810145


In [25]:
y_pred=lr.predict(features_test)
f1 = f1_score(labels_test, y_pred, average='macro')
print("F1 score: ",f1)

F1 score:  0.8484557893282343


In [26]:
y_pred=rfc.predict(features_test)
f1 = f1_score(labels_test, y_pred, average='macro')
print("F1 score: ",f1)

F1 score:  0.7170242526650754


# Chương trình 

In [27]:
print("Nhap: ")
ch=(str(input()))
ch = re.sub(r'[^a-zA-Z]',' ',ch)
ch_tokens = word_tokenize(ch)
ch_sentence = [w for w in ch_tokens if not w in stop_words]

Nhap: 
ARSENAL stars Emiliano Martinez and Lucas Torreira celebrated their FA Cup triumph with a late-night trip to McDonalds.  The Gunners won the competition for a record 14th time thanks to two goals from captain Pierre-Emerick Aubameyang.  Lucas Torreira uploaded this photo of him and team-mate Emiliano Martinez grabbing some late-night grub 3 Lucas Torreira uploaded this photo of him and team-mate Emiliano Martinez grabbing some late-night grubCredit: Instagram / @ltorreira34 Christian Pulisic, who was forced off during the game with a hamstring injury, scored what proved to be a consolation for Chelsea.  Unsurprisingly players and fans partied in the aftermath of a unique triumph at Wembley.  Due to the coronavirus, the game was played behind-closed-doors at the national stadium.  Martinez and Torreira were snapped getting some late-night grub at a Mcdonalds.  ¡ C A M P E O N E S ! 🏆#HeadsUpFACupFinal | @EmiratesFACup pic.twitter.com/Wd9l52zeQt  — 🎗 Arsenal (@Arsenal) August 1, 2

In [28]:
ch_val = tfIdfVec.transform([ch]).toarray()

In [29]:
ch_pred=lsvc.predict(ch_val)

In [30]:
if (ch_pred[0]==0):
  print('femail')
if (ch_pred[0]==1):
  print('films')
if (ch_pred[0]==2):
  print('health')
if (ch_pred[0]==3):
  print('lifestyle')
if (ch_pred[0]==4):
  print('sport')
if (ch_pred[0]==5):
  print('technology')
if (ch_pred[0]==6):
  print('travel')
if (ch_pred[0]==7):
  print('tvshowbiz')

sport
